<a href="https://colab.research.google.com/github/wdnam106/hello-world/blob/master/competition_supervised1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle #install kaggle API
# kaggle Token on Google drive
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [3]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /content/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

!kaggle competitions download -c pubg-finish-placement-prediction -p /content 
!unzip \*.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
- path is now set to: {/content}
sample_submission_V2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_V2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_V2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sample_submission_V2.csv.zip
replace sample_submission_V2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission_V2.csv  

Archive:  test_V2.csv.zip
replace test_V2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test_V2.csv             y


Archive:  train_V2.csv.zip
replace train_V2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: train_V2.csv            y
y


3 archives were successfully processed.


In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tr_train = pd.read_csv('train_V2.csv')
tr_test = pd.read_csv('test_V2.csv')


In [0]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [43]:
tr_train = reduce_mem_usage(tr_train)

Memory usage of dataframe is 1017.83 MB
Memory usage after optimization is: 322.31 MB
Decreased by 68.3%


In [44]:
tr_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int8
boosts             int8
damageDealt        float16
DBNOs              int8
headshotKills      int8
heals              int8
killPlace          int8
killPoints         int16
kills              int8
killStreaks        int8
longestKill        float16
matchDuration      int16
matchType          object
maxPlace           int8
numGroups          int8
rankPoints         int16
revives            int8
rideDistance       float16
roadKills          int8
swimDistance       float16
teamKills          int8
vehicleDestroys    int8
walkDistance       float16
weaponsAcquired    int16
winPoints          int16
winPlacePerc       float16
dtypes: float16(6), int16(5), int8(14), object(4)
memory usage: 322.3+ MB


In [0]:
tr_train = tr_train.dropna()
tr_train_sample = tr_train.sample(n=10000, random_state = 76)
XX = tr_train_sample[['killPlace','walkDistance']]
YY = tr_train_sample['winPlacePerc']


In [0]:
from sklearn.preprocessing import scale
from sklearn import preprocessing
X_scaled = scale(XX)
lab_enc = preprocessing.LabelEncoder()
YY_encoded = lab_enc.fit_transform(YY)

In [37]:
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

logreg = linear_model.LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X_scaled, YY_encoded, test_size = 0.3, random_state=55)
logreg.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [0]:
y_pred = logreg.predict(X_test)

In [39]:
logreg.score(X_test, y_test)

0.7454612103495766

In [40]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))




120.43736152605278
23945.08408533699


In [11]:
from sklearn.metrics import roc_curve
y_pred_prob = logreg.predict_proba(X_test) [:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.plot([0,1], [0,1],'k--')
plt.plot(fpr, tpr, label = 'Logictic Regression')
plt.show

ValueError: ignored

In [0]:
from sklearn.preprocessing import scale